In [1]:
## Standard Imports ##
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import numpy as np
import scipy as sp

import time as timeit
import datetime
import sys,os

## Particle Filter Imports ##
import particles
from particles import state_space_models as ssm
from particles import smc_samplers as ssp
from particles import distributions as dists
import warnings; warnings.simplefilter('ignore')  # hide warnings 

## Compiling the Particle Filter Example
Referencing the description to ```03-PF_Example```, the particle filter scheme used in this illustrative study developes a convergent posterior model for the majority of cases using the parameters ```init_Nx = 10``` and ```N = 50```. For those cases which are not convergent under those conditions, the parameters ```init_Nx = 15``` and ```N = 75``` were used. 

The purpose of this code is then to combine the outputs from these disparate starting conditions to develop a complete example case which includes results for all 50 prior distributions on the parameters. 

## Compile Results

In [ ]:
### Lay Out The Files Holding the PF Results ###
pfOutFiles = np.array(['../04-Data/Linear/outputPF_Nx10_Ntheta50.npz',
                       '../04-Data/Linear/outputPF_Nx15_Ntheta75.npz'])
saveFile = '02 - Results/outputPF'

### Generate Storage Over Inferred States/Parameters ###
outData = np.load(pfOutFiles[0])
muHist = np.zeros(outData['muHist'].shape)
    # mean of the inferred parameters for each inference trial
    # over the observation period. This is what the EKF directly
    # outputs
stdHist = np.zeros(outData['stdHist'].shape)
    # standard deviation of the inferred parameters for each 
    # inference trial over the observation period. This is 
    # what the EKF directly outputs
meanHist = np.zeros(outData['meanHist'].shape)
    # mean of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modeHist = np.zeros(outData['modeHist'].shape)
    # mode of the inferred states and the underlying parameters
    # for each inference trial over the observation period. This
    # measure transforms the parameters to a lognormal distribution
    # such that the statistics on the true parameter values can be
    # extracted. 
modStates = np.zeros(outData['modStates'].shape)
    # Response history of the inferred system given the input
    # excitation. Essentially, we're remodeling the behavior of 
    # the system given our selections on point estimates of the 
    # parameters from the posterior. 
runTime = np.zeros(outData['runTime'].shape)
    # Computational time for each inference trial. 
Nx = np.zeros(outData['Nxs'].shape)
    # Tracks the increase in number of particles on the parameters
    # over the inference history. 

### Combine Results Files ###
for i in range(len(pfOutFiles)):
    outData = np.load(pfOutFiles[i])

    for j in range(outData['muHist'].shape[0]):
        ## If the trial failed to converge, do not include in compiled results
        if outData['Nxs'][j,-1] == 0:
            continue
        ## If the trial converged, include in compiled results
        else:
            print('Inserting Data from ' + pfOutFiles[i] + '\n\t@ Iteration %d:'%(j))

            muHist[j,:,:] = outData['muHist'][j,:,:]         # inference history of untransformed state/par means
            stdHist[j,:,:] = outData['stdHist'][j,:,:]       # inference history of state/par standard deviations
            meanHist[j,:,:] = outData['meanHist'][j,:,:]     # inference history of transformed state/par means
            modeHist[j,:,:] = outData['modeHist'][j,:,:]     # inference history of transformed state/par modes
            modStates[j,:,:] = outData['modStates'][j,:,:]   # states that have been remodeled based on the final modes of the parameters
            Nx[j,:] = outData['Nxs'][j,:]                  # Number of particle sample runs
            runTime[j] = outData['runTime'][j]           # Run Time

            print('\tComputation Time = %d minutes and %d seconds' %(np.floor(runTime[j]), 
                                                            np.floor((runTime[j]*60) - 60*np.floor(runTime[j]))) )
            print('\tMode of Final Parameter Distributions: \n\t\txi = %.4f,\n\t\twn = %.4f\n'
              %(modeHist[j,2,-1],modeHist[j,3,-1]))
            
### Save Results ###
np.savez(saveFile, muHist = muHist,stdHist=stdHist, meanHist=meanHist, modeHist=modeHist, 
         modStates = modStates, Nx = Nx, runTime = runTime)